In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9a7ac9c7db5ebaa1eebe76160d4dc44a7a8b9c8f89da7d05f72dae0ecbdd8ab1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import col, max, min, avg

spark = SparkSession.builder.master('local[*]').\
        appName('alex').getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
        
sc = spark.sparkContext

sc.addFile('/content/drive/MyDrive/transactions_amostra.csv')

# carregando o arquivo
df_tde = spark.read.csv("file://" + SparkFiles.get('transactions_amostra.csv'), sep=";", header=True, inferSchema=True)

In [4]:
# mostrando a base
df_tde

country_or_area,year,comm_code,commodity,flow,trade_usd,weight_kg,quantity_name,quantity,category
Belgium,2016,920510,Brass-wind instru...,Export,571297,3966.0,Number of items,4135.0,92_musical_instru...
Guatemala,2008,660200,"Walking-sticks, s...",Export,35022,5575.0,Number of items,10089.0,66_umbrellas_walk...
Barbados,2006,220210,"Beverage waters, ...",Re-Export,81058,44458.0,Volume in litres,24113.0,22_beverages_spir...
Tunisia,2016,780411,Lead foil of a th...,Import,4658,121.0,Weight in kilograms,121.0,78_lead_and_artic...
Lithuania,1996,560110,"Sanitary towels, ...",Export,76499,5419.0,Weight in kilograms,5419.0,56_wadding_felt_n...
Denmark,2011,310100,Animal or vegetab...,Export,4903675,1.902844E7,Weight in kilograms,1.902844E7,31_fertilizers
Thailand,1994,920290,String musical in...,Import,2088672,0.0,Number of items,59595.0,92_musical_instru...
Portugal,2004,511119,"Woven fabric, >85...",Export,1546575,87367.0,Weight in kilograms,87367.0,51_wool_animal_ha...
Congo,2011,420690,"Articles of gut, ...",Export,883,9.0,Weight in kilograms,9.0,42_articles_of_le...
Antigua and Barbuda,2016,620332,"Mens, boys jacket...",Export,12988,1403.0,Number of items,648.0,62_articles_of_ap...


In [5]:
# Questão1 
# 1. (Easy) The number of transactions involving Brazil;

# primeiramente fazemos um filter.like("Brazil") para pegar todas as linhas em que o 'Brazil' se encontra passando a coluna especifica col('country_or_area'), depois fazemos um .count() para contar todas as ocorrencias
df_tde.filter(col('country_or_area').like('Brazil')).count()

54762

In [6]:
# Questão2
# 2. (Easy) The number of transactions per year;

# primeiramente fazemos um groupBy na coluna 'year' passando pelos anos e um .count() para contar as ocorrencias
df_tde.groupBy('year').count().show(10)

+----+------+
|year| count|
+----+------+
|1990| 18188|
|2003|101125|
|2007|105891|
|2015|101973|
|2006|109186|
|2013|104734|
|1997| 77876|
|1988|  8568|
|1994| 54143|
|2014|103231|
+----+------+
only showing top 10 rows



In [7]:
# Questão3 
# 3. (Easy) The number of transactions per flow type and year;

# fazemos um groupBy nas colunas year e flowtype e um .count() para contar as ocorrencias
df_tde.groupBy('year', 'flow').count().show(10)

+----+---------+-----+
|year|     flow|count|
+----+---------+-----+
|2011|   Import|62270|
|2006|   Import|64345|
|1994|Re-Export| 2094|
|2009|   Export|35489|
|1998|   Import|49870|
|2012|   Export|35241|
|1999|Re-Export| 4049|
|2006|   Export|36981|
|2015|Re-Import| 3005|
|2014|Re-Import| 2902|
+----+---------+-----+
only showing top 10 rows



In [8]:
#Questão 4
# 4. (Easy) The average of commodity values per year;

# fazemos um groupBy na coluna ano e tiramos a média da coluna "trade_usd"
df_tde.groupBy('year').avg('trade_usd')

year,avg(trade_usd)
1990,1.3192706620958874E7
2003,1.421775659984178E7
2007,2.58920247040825E7
2015,3.18705453775215E7
2006,2.309561335510047E7
2013,4.000417948798862E7
1997,1.0552287985399866E7
1988,1.6750924673436042E7
1994,1.1536598545389062E7
2014,4.07714802568802E7


In [9]:
# Questão 5
# (Easy) The average price of commodities per unit type, year, and category in the export flow in Brazil;

# primeiramente fazemos um filter na coluna country_or_area pegando o pais Brazil. 
# Depois fazemos um novo filter para pegar o flow type do tipo Export 
# pegamos as colunas de quantity_name, year, category, flow e country_or_area
# por fim fazemos a media na coluna trade_usd

df_tde.filter(col("country_or_area").like("Brazil")).filter(col("flow").like("Export")).groupBy('quantity_name', 'year', 'category', 'flow', 'country_or_area').avg('trade_usd')

quantity_name,year,category,flow,country_or_area,avg(trade_usd)
Weight in kilograms,2011,48_paper_paperboa...,Export,Brazil,4.42723402E7
Number of items,2000,62_articles_of_ap...,Export,Brazil,443135.4074074074
Weight in kilograms,2011,63_other_made_tex...,Export,Brazil,786739.0
Weight in kilograms,1995,16_meat_fish_and_...,Export,Brazil,591473.0
Weight in kilograms,2011,44_wood_and_artic...,Export,Brazil,4.3177107083333336E7
Number of items,2010,88_aircraft_space...,Export,Brazil,4.7883109E8
Weight in kilograms,2013,27_mineral_fuels_...,Export,Brazil,5.067649188888889E8
Weight in kilograms,1996,72_iron_and_steel,Export,Brazil,4.668934694E7
Weight in kilograms,2000,91_clocks_and_wat...,Export,Brazil,36086.0
Volume in litres,1989,20_vegetable_frui...,Export,Brazil,8978975.0


In [10]:
# Questao 6 
# (Medium) The maximum, minimum, and mean transaction price per unit type and year;

# primeiramente fazemos um groupBy passando quantity_name e year como referencia.
# depois usamos um agg que é usado para retornar varias variaveis, onde passamos o valor maximo, o valor minimo e a média da coluna trade_usd

df_tde.groupBy('quantity_name', 'year').agg(max('trade_usd'), min('trade_usd'), avg('trade_usd'))

quantity_name,year,max(trade_usd),min(trade_usd),avg(trade_usd)
Electrical energy...,1994,16549948,588,4122301.5
Number of pairs,2008,8231914307,23,3.9991725244176015E7
Weight in carats,2013,1156063734,487,9.718167391666667E7
Thousands of items,2006,27188639,58,6971520.423076923
Volume in cubic m...,2005,740206429,6,2.1613620009375E7
Area in square me...,2013,3465872352,7,1.0891966478880407E7
Length in metres,1996,558384790,13,1.493895061967213E7
Number of pairs,2013,3087358939,15,3.524038769922879E7
Volume in litres,1988,18423300970,6,8.903355118110237E7
Number of pairs,1990,119229449,130,6225074.855421687


In [11]:
# Questao 7
# (Hard) The most commercialized commodity (summing the quantities) in 2016, per flow type.

# primeiramente filtramos a coluna ano por 2016 e fazemos um groupBy por flow e commodity junto com a soma da coluna quantity, por fim mudamos o nome da coluna
# no df2 pegamos o valor maximo presente na coluna quantity, e mudamos novamente o nome da coluna.
# e por fim usando um comando alias retornamos o flow, a commodity e a quantity (valor maximo de cada flow type e commodity)



df1 = df_tde.filter(col("year").like("2016")).groupBy('flow', 'commodity').sum('quantity').withColumnRenamed('sum(quantity)', 'quantity')
df2 = df1.groupBy(df_tde.flow).max('quantity').withColumnRenamed('max(quantity)', 'quantity')
df2.alias("df2").join(df1.alias("df1"), df1.quantity == df2.quantity).select("df2.flow", "df1.commodity", "df2.quantity")

flow,commodity,quantity
Import,"Iron ore, concent...",1.073469712878E12
Export,"Ice, snow and pot...",6.99847369665E11
Re-Export,Safety razor blad...,1.261968E9
Re-Import,"Chem wood pulp, s...",3.8774873E7
